In [1]:
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
url = 'https://www.tripadvisor.com.sg/'
driver = webdriver.Chrome(executable_path='/Users/user/Documents/GitHub/Just Analytics/chromedriver/chromedriver_1')

In [3]:
## Navigating throught the homepage of TripAdvisor to search for hotels in Singapore
driver.get(url)

search_hotel = driver.find_element_by_class_name('typeahead_input')

search_hotel.send_keys('Singapore')

driver.find_element_by_class_name('submit_text').click()

In [9]:
## Get the web elements for each hotel to access later on

hotel = driver.find_elements_by_class_name('property_title')

In [10]:
## Preview of list
hotel[:3]

[<selenium.webdriver.remote.webelement.WebElement (session="f00b719ed4428a93cad6dbe0a9a9c483", element="0.12083207460701839-1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f00b719ed4428a93cad6dbe0a9a9c483", element="0.12083207460701839-2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f00b719ed4428a93cad6dbe0a9a9c483", element="0.12083207460701839-3")>]

In [11]:
## Default handle is main page with listing of hotels
default_handle = driver.current_window_handle
hotel_list = {}

## Iterates through each hotel for its reviews
for i in range(len(hotel)):
    hotel[i].click()
    
    driver.switch_to_window(driver.window_handles[1])
    
    HTML = driver.page_source
    soup = BeautifulSoup(HTML,'lxml')
    
    hotel_name = soup.find('h1',{'id':'HEADING'}).text ## Gets hotel's name
    hotel_list[hotel_name] = []
    
    reviews = soup.find_all('div',{'class':'review-container'}) ## Finding all review containers
    
    for j in range(len(reviews)):
        this_review = {}
        this_review['Subject'] = reviews[j].find('span',{'class':'noQuotes'}).text ## subject
        this_review['Review'] = reviews[j].find('p',{'class':'partial_entry'}).text ## review
        this_review['User'] = reviews[j].find('div',{'class':'info_text'}).find('div').text ## user
        this_review['Time'] = reviews[j].find('span',{'class':'ratingDate'}).text ## Time
        try:
            this_review['Location'] = reviews[j].find('div',{'class':'userLoc'}).text ## Location
        except:
            continue
        hotel_list[hotel_name].append(this_review)
    
    driver.close()
    driver.switch_to_window(default_handle)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use driver.switch_to.window instead
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: use driver.switch_to.window instead


In [12]:
## List of hotels with their reviews

hotel_list 

{'Carlton Hotel Singapore': [{'Location': 'London, United Kingdom',
   'Review': "I was in the higher rate tower . The room was quite OK and relatively spacious but not as spacious as the Swissotel over the road. Nevertheless, the quality of this hotel is somewhat superior to it's neighbours!\n\nI was there for an overnight stay...More",
   'Subject': 'A good hotel in a central position with pretty good city views',
   'Time': 'Reviewed yesterday ',
   'User': 'Michael T'},
  {'Location': 'Singapore',
   'Review': 'Recently attended an event hosted at the function rooms. \n\nVery good service, food and drink provisions; love the breakout areas outside the function rooms - the seats and tables. Good planning. \n\nQuality of the food was exceptional, especially with more than 500 participants.',
   'Subject': 'Fantastic Event experience',
   'Time': 'Reviewed yesterday ',
   'User': 'Ahguru'},
  {'Location': 'Brisbane, Australia',
   'Review': 'Lovely hotel, but the reception staff need 

In [19]:
df = pd.DataFrame(hotel_list['PARKROYAL on Pickering'])

In [20]:
df.to_json('PARKROYAL_review')

In [21]:
import pickle

In [23]:
pickle.dump(hotel_list,open('hotel_list_reviews','wb'))